### Import

In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

### GPU

In [2]:
torch.cuda.device_count()

cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')

device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

### INIT

In [3]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Mono'
excl_folders = ['NoFX_mono', 'NoFX_mono_preprocessed']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=6

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, shuffle=True)

# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

In [4]:
print(len(dataset))
print(dataset.mel_shape)
print(dataset.num_fx)
print(dataset.max_num_settings)
dataset.fx_to_label

163488
torch.Size([1, 128, 87])
14
6


{'808': 0,
 'BD2': 1,
 'BMF': 2,
 'DPL': 3,
 'DS1': 4,
 'FFC': 5,
 'MGS': 6,
 'MT2': 7,
 'OD1': 8,
 'RAT': 9,
 'RBM': 10,
 'SD1': 11,
 'TS9': 12,
 'VTB': 13}

### TRAIN FxNET and SetNET

In [5]:
# model
fxnet = models.FxNet(n_classes=dataset.num_fx).to(device)
# optimizer
optimizer_fxnet = optim.Adam(fxnet.parameters(), lr=0.001)
# loss function
loss_func_fxnet = nn.CrossEntropyLoss()

# model
setnet = models.SettingsNetCond(n_settings= dataset.max_num_settings,
                                mel_shape=dataset.mel_shape, 
                                num_embeddings=dataset.num_fx, 
                                embedding_dim=50)
# optimizer
optimizer_setnet = optim.Adam(setnet.parameters(), lr=0.001)
# loss function
loss_func_setnet = nn.MSELoss(reduction='mean')

In [6]:
print(fxnet)
print(setnet)

FxNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=14, bias=True)
)
SettingsNetCond(
  (emb): Embedding(14, 50)
  (fc0): Linear(in_features=50, out_features=11136, bias=True)
  (conv1): Conv2d(2, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=6, bias=True)
)


In [7]:
# TRAIN and TEST FxNet and SettingsNetCond OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainer.train_cond_nets(
        model_fx=fxnet, 
        model_set=setnet,
        optimizer_fx=optimizer_fxnet, 
        optimizer_set=optimizer_setnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch,
        loss_function_fx=loss_func_fxnet,
        loss_function_set=loss_func_setnet, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_cond_nets(
        model_fx=fxnet, 
        model_set=setnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler,
        loss_function_fx=loss_func_fxnet, 
        loss_function_set=loss_func_setnet, 
        device='cpu'
    )
    
    test_loss, test_correct, test_results = trainer.test_cond_nets(
        model_fx=fxnet, 
        model_set=setnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler,
        loss_function_fx=loss_func_fxnet, 
        loss_function_set=loss_func_setnet, 
        device='cpu'
    )

    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

t: 9699/32698
		Percentage Correct: 27.53	Percentage Fx Correct: 87.66	Percentage Set Correct: 29.66
Train Epoch: 8	[1000/117711 (1%)]	Total Loss: 2.4333	Avg Loss: 0.0024
Train Epoch: 8	[2000/117711 (2%)]	Total Loss: 4.3744	Avg Loss: 0.0022
Train Epoch: 8	[3000/117711 (3%)]	Total Loss: 6.3009	Avg Loss: 0.0021
Train Epoch: 8	[4000/117711 (3%)]	Total Loss: 8.5812	Avg Loss: 0.0021
Train Epoch: 8	[5000/117711 (4%)]	Total Loss: 10.5272	Avg Loss: 0.0021
Train Epoch: 8	[6000/117711 (5%)]	Total Loss: 12.9479	Avg Loss: 0.0022
Train Epoch: 8	[7000/117711 (6%)]	Total Loss: 14.9852	Avg Loss: 0.0021
Train Epoch: 8	[8000/117711 (7%)]	Total Loss: 16.8073	Avg Loss: 0.0021
Train Epoch: 8	[9000/117711 (8%)]	Total Loss: 18.6766	Avg Loss: 0.0021
Train Epoch: 8	[10000/117711 (8%)]	Total Loss: 20.5780	Avg Loss: 0.0021
Train Epoch: 8	[11000/117711 (9%)]	Total Loss: 22.4455	Avg Loss: 0.0020
Train Epoch: 8	[12000/117711 (10%)]	Total Loss: 24.3267	Avg Loss: 0.0020
Train Epoch: 8	[13000/117711 (11%)]	Total Loss:

In [8]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  60.997697751272185
Epoch:  46

Accuracy:  57.07622906950073
Epoch:  40

Accuracy:  57.31543213652211
Epoch:  40



In [11]:
# SAVE
models_folder = 'saved_models'
model_name_fx = '20201001_fxnet'
model_name_set = '20201001_setnetcond'
results_folder = 'saved_results'
results_subfolder = model_name_fx + '_and_setnetcond'

In [12]:
# SAVE MODEL
torch.save(fxnet, '%s/%s' % (models_folder, model_name_fx))
torch.save(setnet, '%s/%s' % (models_folder, model_name_set))

In [14]:
# SAVE RESULTS - all losses, all correct, best results
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
best_train_results_npy = np.array(all_train_results[40])

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
best_val_results_npy = np.array(all_val_results[40])

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
best_test_results_npy = np.array(all_test_results[40])

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_losses_fx')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_correct_fx')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_train_results_fx')), arr=best_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_losses_fx')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_correct_fx')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_val_results_fx')), arr=best_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_losses_fx')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_correct_fx')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_test_results_fx')), arr=best_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'fx_labels')), arr=fx_labels_npy)